In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import os
import pretty_midi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from pydub import AudioSegment

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
def load_midi_files(directory):
    notes = []
    for filename in os.listdir(directory):
        if filename.endswith(".mid"):
            midi_data = pretty_midi.PrettyMIDI(os.path.join(directory, filename))
            for instrument in midi_data.instruments:
                if instrument.is_drum:
                    continue
                for note in instrument.notes:
                    pitch = note.pitch
                    duration = note.end - note.start
                    notes.append((pitch, duration))
    return notes


In [4]:
data_directory = "./dataset/"
notes = load_midi_files(data_directory)
pitchnames = sorted(set(note[0] for note in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [5]:
sequence_length = 100
n_vocab = len(pitchnames)
network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[note[0]] for note in sequence_in])
    network_output.append(note_to_int[sequence_out[0]])

n_patterns = len(network_input)
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = tf.keras.utils.to_categorical(network_output)